1.	How can a list of hosts in a group be looped over within a template?

This can be done by accessing the “$groups” dictionary in the template, like so:

{% for host in groups['db_servers'] %}
{{ host }}
{% endfor %}
If we need to access facts also we need to make sure that the facts have been populated. For instance, a play that talks to db_servers:

- hosts: db_servers
tasks:
- debug: msg="Something to debug"
Now, this can be used within a template, like so:

{% for host in groups['db_servers'] %}
{{ hostvars[host]['ansible_eth0']['ipv4']['address'] }}
{% endfor %}.


2.	What is Ansible's ad-hoc command?

Ad hoc commands are commands which can be run individually to perform quick functions. These commands need not be performed later. For example, you have to reboot all your company servers. For this, you will run the Adhoc commands from '/usr/bin/ansible'.

3.	How do I set up Nginx using the Ansible playbook?

---
- hosts: all
  tasks:
    - name: ensure nginx is at the latest version
      apt: name=nginx state=latest
      become: yes
    - name: start nginx
      service:
          name: nginx
          state: started
      become: yes
    - name: copy the nginx config file and restart nginx
      copy:
        src: /home/static_site.cfg
        dest: /etc/nginx/sites-available/static_site.cfg
      become: yes
    - name: create symlink
      file:
        src: /etc/nginx/sites-available/static_site.cfg
        dest: /etc/nginx/sites-enabled/default
        state: link
      become: yes
    - name: copy the content of the web site
      copy:
        src: /home/static-site-src/
        dest: /home/static-site
    - name: restart nginx
      service:
        name: nginx
        state: restarted
      become: yes
     

4.	How do I programmatically access the name of a variable?

{{ hostvars[inventory_hostname]['ansible_' + which_interface]['ipv4']['address'] }}

5.	How do Ansible and Puppet vary from one other?

In Puppet, the client pulls configurations from the server, whereas in Ansible, the server pushes configurations to the nodes, for instantaneous deployment. Plus, in Puppet, you have to write the configurations in Puppet's language, as mentioned, whereas in Ansible you use YAML, which is close to English.

6.	What is the purpose of Ansible Tower, and what are its characteristics?

Ansible Tower is the enterprise version of Ansible. It allows sysadmins to deploy all of the benefits of Ansible at scale. And, like Ansible, it integrates with a broad base of your existing technology infrastructure: networking, security, application deployment, storage, software development lifecycle processes, etc.

7.	Describe how you'll recursively copy files to a destination host.

The “copy” module has a recursive parameter. However, if you want this to perform more efficient for a large number of files, then “synchronize” module is the best option for you.

- synchronize:
   src: /first/absolute/path
   dest: /second/absolute/path
   delegate_to: "{{ inventory_hostname }}"

8.	What is the most effective method for making content reusable and redistributable?

To make content reusable and redistributable Ansible roles can be used. Ansible roles are basically a level of abstraction to organize playbooks. For example, if we need to execute 10 tasks on 5 systems, writing all of them in the playbook might lead to blunders and confusion. Instead we create 10 roles and call them inside the playbook.

9.	What are handlers, and what do they do?

Handlers are like special tasks which only run if the Task contains a “notify” directive. 

tasks:
  - name: install nginx
    apt: pkg=nginx state=installed update_cache=true
    notify:
     - start nginx
 handlers:
   - name: start nginx
     service: name=nginx state=started


10.	How can a user module generate encrypted passwords?

ansible all -i localhost, -m debug -a "msg={{ 'mypassword' | password_hash('sha512', 'mysecretsalt') }}"

11.	What is the difference between dot notation and array notation for variables?

Dot notation works fine unless we stump upon few special cases such as

If the variable contains a dot(.), colon(:), starting or ending with an underscore or any known public attribute.
If there’s a collision between methods and attributes of python dictionaries.
Array notation also allows for dynamic variable composition.


12.	What is the purpose of the Ansible synchronize module?

Ansible synchronize is a module similar to rsync in Linux machines which we can use in playbooks. The features are similar to rsync such as archive, compress, delete, etc but there are few limitations also such as

Rsync must be installed on both source and target systems
Need to specify delegate_to to change the source from localhost to some other port
Need to handle user permission as files are accessible as per remote user.
We should always give the full path of the destination host location in case we use sudo otherwise files will be copied to the remote user home directory.
Linux rsync limitations related to hard links are also applied here.
It forces -delay-updates to avoid the broken state in case of connection failure

13.	What is the purpose of the Ansible firewalld module?

Ansible firewalld is used to manage firewall rules on host machines. This works just as Linux firewalld daemon for allowing/blocking services from the port. It is split into two major concepts

Zones: This is the location for which we can control which services are exposed to or a location to which one the local network interface is connected.
Services: These are typically a series of port/protocol combinations (sockets) that your host may be listening on, which can then be placed in one or more zones

14.	What distinguishes the Ansible set fact module from vars, vars file, and include var?

 In Ansible, set_fact is used to set new variable values on a host-by-host basis which is just like ansible facts, discovered by the setup module. These variables are available to subsequent plays in a playbook. In the case of vars, vars_file, or include_var we know the value beforehand whereas when using set_fact, we can store the value after preparing it on the fly using certain tasks like using filters or taking subparts of another variable. We can also set a fact cache over it.

set_fact variable assignment is done by using key-pair values where the key is the variable name and the value is the assignment to it. A simple example will be like below

- set_fact:
one_fact: value1
second_fact:
value2

15.	When is it risky to use a variable to bulk-set task arguments?

All of the task's arguments can be dictionary-typed variables which can be useful in some dynamic execution scenarios also. However, Ansible issues a warning since it introduces a security risk.